In [19]:
from selenium.webdriver import (Chrome)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = 35

In [20]:
df = pd.read_csv('../data-housing/everett-kent-may-7.csv')

In [21]:
df = df.rename(columns={'URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)': 
                        'URL'})
col = []
browser = Chrome()
for idx, row in df.iterrows():
    url = row['URL']
    try:
        browser.get(url)
        sel = "div.sectionContent"
        description = browser.find_elements_by_css_selector(sel)
        col.append(description[0].text)
    except:
        col.append('Invalid URL')
#     time.sleep(3 + random.random() * 10)
    if idx % 4 == 0:
        print(idx) 
df['DESC'] = pd.Series(col)

0
4
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
128
132
136
140
144
148
152
156
160
164
168
172
176
180
184
188
192
196
200
204
208
212
216
220
224
228
232
236
240
244
248
252
256
260
264
268
272
276
280
284
288
292
296
300
304
308
312
316
320
324
328
332
336
340
344
348


In [24]:
df.drop(['SOLD DATE', 'FAVORITE', 'INTERESTED', 'NEXT OPEN HOUSE START TIME', 'NEXT OPEN HOUSE END TIME', 'SOURCE', 'MLS#'], axis=1, inplace=True)

In [25]:
df.head()

,SALE TYPE,PROPERTY TYPE,ADDRESS,CITY,STATE,ZIP,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,URL,LATITUDE,LONGITUDE,DESC
0,MLS Listing,Single Family Residential,711 153rd St SW,Lynnwood,WA,98087,750000,4,3.00,Lynnwood,3252.0,7405.0,2016.0,3,231.0,NaN,Active,http://www.redfin.com/WA/Lynnwood/711-153rd-St...,47.859744,-122.243075,Gorgeous 4 bed 3 bath Lynnwood home w/ high-en...
1,MLS Listing,Single Family Residential,4736 S 164th St,Tukwila,WA,98188,460000,3,1.00,Tukwila,1480.0,18160.0,1960.0,3,311.0,NaN,Active,http://www.redfin.com/WA/Tukwila/4736-S-164th-...,47.457014,-122.273428,Head down the driveway to the privacy of this ...
2,MLS Listing,Townhouse,4123 178th Lane SE #2,Bellevue,WA,98008,469000,2,2.25,South Lake Sammamish,962.0,114792.0,1988.0,3,488.0,318.0,Active,http://www.redfin.com/WA/Bellevue/4123-178th-L...,47.570999,-122.102356,Gorgeous 2Bed 2.25Bath Townhome w/ Stunning Vi...
3,MLS Listing,Single Family Residential,29003 55th Ave S,Auburn,WA,98001,699950,5,2.25,West Hill,2750.0,49813.0,1971.0,3,255.0,NaN,Active,http://www.redfin.com/WA/Auburn/29003-55th-Ave...,47.341607,-122.266415,Rare opportunity for this very unique home! Th...
4,MLS Listing,Single Family Residential,9252 11th Ave NW,Seattle,WA,98117,725000,3,1.75,Crown Hill,2060.0,7806.0,1942.0,3,352.0,NaN,Active,http://www.redfin.com/WA/Seattle/9252-11th-Ave...,47.697541,-122.369796,"Situated on lg corner lot, meticulously mainta..."


In [26]:
df.to_csv('../data-housing/training-data.csv')

In [28]:
df['ZIP'] = df['ZIP'].astype('category')

In [29]:
pwd

'/Users/elisereppond/galvanize/capstone/housing-recommender'